Composite nodes
=====================

In [1]:
using ForneyLab

# Define new node type called StateTransition, with exposed variables called (x, x_prev, y):
@composite StateTransition (x, x_prev, y) begin
    x ~ GaussianMeanVariance(x_prev, constant(1.0))
    y ~ GaussianMeanVariance(x, constant(1.0))
end

g = FactorGraph()

# Build factor graph
x_prev ~ GaussianMeanVariance(constant(0.0), constant(1.0))
y = Variable(id=:y)
x = Variable(id=:x)
StateTransition(x, x_prev, y)
placeholder(y, :y)

# Assign id for easy lookup
x.id = :x
;

In [2]:
# Define custom rule for sum-product message towards x
@sumProductRule(:node_type     => StateTransition,
                :outbound_type => Message{Gaussian},
                :inbound_types => (Void, Message{Gaussian}, Message{PointMass}),
                :name          => SPStateTransitionVGP)

# Build algorithm
schedule = sumProductSchedule(x)

ForneyLab.draw(g, schedule=schedule)

<?xml version="1.0" encoding="UTF-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
<!-- Generated by graphviz version 2.36.0 (20140111.2315)
 -->
<!-- Title: G Pages: 1 -->
 
 
 G 
 
<!-- 15990764942943378002 -->
 15990764942943378002 
 
 clamp_2 
 
<!-- 9576424178951761650 -->
 9576424178951761650 
 
 clamp_1 
 
<!-- 17891044155864513250 -->
 17891044155864513250 
 
 𝒩 
 gaussian_1 
 
<!-- 17891044155864513250--15990764942943378002 -->
 17891044155864513250--15990764942943378002 
 
 1 out 
 2 variance 
 
<!-- 17891044155864513250--9576424178951761650 -->
 17891044155864513250--9576424178951761650 
 
 1 out 
 1 mean 
 
<!-- 16188645096571823805 -->
 16188645096571823805 
 
 placeholder_y 
 
<!-- 16898212529421097029 -->
 16898212529421097029 
 
 StateTransition 
 statetransition_1 
 
<!-- 16188645096571823805--16898212529421097029 -->
 16188645096571823805--16898212529421097029 
 
 3 y 
 1 out 
 
<!-- 16898212529421097029--17891044155864513250 -->
 16898212529421097029--17891044155864513250 
 
 3 out 
 (1) 
 2 x_prev 
 
<!-- 122680975300518941912 -->
 122680975300518941912 
 
<!-- 122680975300518941912--16898212529421097029 -->
 122680975300518941912--16898212529421097029 
 
 1 x 
 (2)

In [3]:
# Implement rule to send some dummy message for now
ruleSPStateTransitionVGP(::Void, ::Message{Gaussian}, ::Message{PointMass}) = Message(Gaussian, m=2.0, v=3.0)

algo = messagePassingAlgorithm(schedule, x)

# Inspect Julia code of algorithm
println(algo)

function step!(data::Dict, marginals::Dict=Dict(), messages::Vector{Message}=Array{Message}(2))

messages[1] = ruleSPGaussianMeanVariancePPV(Message(PointMass, m=0.0), Message(PointMass, m=1.0), nothing)
messages[2] = ruleSPStateTransitionVGP(nothing, messages[1], Message(PointMass, m=data[:y]))

marginals[:x] = messages[2].dist

return marginals

end


In [4]:
eval(parse(algo))

data = Dict(:y => 1.0)
marginals = step!(data)

println(marginals[:x])

ForneyLab.ProbabilityDistribution{ForneyLab.Gaussian}(Dict(:m=>2.0,:v=>3.0))
